In [16]:
import numpy as np
from dl_toolbox.utils import get_tiles
import shapely
import rasterio

DATA_POLYGON = shapely.Polygon(
    [[359326,4833160],
     [376735,4842547],
     [385238,4826271],
     [367914,4816946],
     [359326,4833160]]
)

def generate_polygon(bbox):
    """
    Generates a list of coordinates: [[x1,y1],[x2,y2],[x3,y3],[x4,y4],[x1,y1]]
    """
    return [[bbox[0],bbox[1]],
             [bbox[2],bbox[1]],
             [bbox[2],bbox[3]],
             [bbox[0],bbox[3]],
             [bbox[0],bbox[1]]]

with rasterio.open("/work/OT/ai4geo/DATA/DATASETS/DIGITANIE/Toulouse/Toulouse.tif") as f:
    h,w = f.shape
    full_in = False
    n = 0
    while not full_in:
        crop = rasterio.windows.Window(n, n, h-2*n, w-2*n)                            
        left, bottom, right, top = rasterio.windows.bounds(
            crop,
            transform=f.transform
        )
        crop_poly = shapely.Polygon(generate_polygon((left, bottom, right, top)))
        inter = shapely.intersection(crop_poly, DATA_POLYGON)
        area = shapely.area(inter) / shapely.area(crop_poly)
        full_in = (area>=.99)
        print(full_in)
        n += 1000
    n -= 1000
    mean = np.array([0,0,0,0], dtype=np.float64)
    i = 1
    for tile in get_tiles(w-2*n, h-2*n, 5000, col_offset=n, row_offset=n, cover_all=False):
        tile_img = f.read(window=tile)
        tile_mean = np.mean(tile_img, axis=(1,2))
        mean += (tile_mean - mean) / i
        i += 1
        print(mean)
    print(mean)
    #print(crop_img.shape)

False
False
False
False
False
False
False
False
False
False
False
False
False
True
[0.09849199 0.10335866 0.11288217 0.21531706]
[0.10470457 0.10885551 0.11775978 0.21399323]
[0.10903376 0.11242765 0.12168191 0.20664583]
[0.10844705 0.11138627 0.12069001 0.20740956]
[0.10557562 0.10966521 0.11909904 0.20698276]
[0.10710744 0.11109315 0.12032198 0.20756214]
[0.11020168 0.11371848 0.1227758  0.2059096 ]
[0.11283427 0.11598756 0.12498699 0.20361528]
[0.11156363 0.11483422 0.12402334 0.20355269]
[0.1106208  0.11421487 0.12348139 0.20388228]
[0.10903827 0.11329056 0.12251379 0.20506695]
[0.10861864 0.1129546  0.12211177 0.20540509]
[0.10802666 0.11251532 0.12180922 0.20359494]
[0.10809964 0.11251282 0.12184742 0.20328178]
[0.10789525 0.11236557 0.12185172 0.20181251]
[0.1078277  0.11238775 0.12180501 0.20202577]
[0.10830327 0.11284892 0.12225782 0.20118916]
[0.10843413 0.11267972 0.12215294 0.19965764]
[0.10836564 0.11236478 0.12190191 0.1976699 ]
[0.10763204 0.1118677  0.12150295 0.1969869

In [15]:
import rasterio
from rasterio.enums import Resampling
from rasterstats import zonal_stats
import shapely
from pathlib import Path
import numpy as np

toulouse = shapely.Polygon(
    [[359326,4833160],
     [376735,4842547],
     [385238,4826271],
     [367914,4816946],
     [359326,4833160]]
)

arcachon = shapely.Polygon(
    [[629473,4959797],
     [649299,4960143],
     [649315,4940864],
     [630504,4940913],
     [629473,4959797]]
)

biarritz = shapely.Polygon(
    [[620357, 4825277],
     [628680, 4825402],
     [629181, 4802997],
     [610093, 4802622],
     [610155, 4808442],
     [620357, 4825277]]
)

brisbane = shapely.Polygon(
    [[494125, 6974741],
     [494125, 6974741],
     [518910, 6973355],
     [518910, 6973355],
     [518712, 6953715],
     [518712, 6953715],
     [493976, 6955348],
     [493976, 6955348],
     [494125, 6974741]]
)

buenosaires = shapely.Polygon(
    [
         [356354,6181219],
         [375609,6181386],
         [376276,6145542],
         [356937,6145459],
         [356354,6181219],
    ]
)

cantho = shapely.Polygon(
    [
         [574413,1117316],
         [593936,1117304],
         [593959,1103770],
         [574344,1103701],
         [574413,1117316],
    ]
)

helsinki = shapely.Polygon(
    [
         [378038,6677645],
         [397619,6676929],
         [397165,6664249],
         [377680,6664846],
         [378038,6677645],
    ]
)

lagos = shapely.Polygon(
    [
         [528578,733313],
         [547696,733545],
         [547542,705913],
         [528578,706068],
         [528578,733313],
    ]
)

cairo = shapely.Polygon(
    [
         [321171,3336141],
         [340490,3335740],
         [340162,3314598],
         [320588,3314926],
         [321171,3336141],
    ]
)

maros = shapely.Polygon(
    [
         [775303,9473671],
         [795021,9473362],
         [794849,9461765],
         [777230,9461971],
         [321171,3336141],
    ]
)

montpellier = shapely.Polygon(
    [
        [569263,4842348],
        [579275,4847307],
        [585825,4841974],
        [585170,4832523],
        [576562,4831868],
        [576936,4825880],
        [579181,4824102],
        [579369,4821576],
        [568982,4816804],
        [569263,4842348],
    ]
)

munich = shapely.Polygon(
    [
         [681767,5348957],
         [701333,5349752],
         [701955,5330290],
         [682666,5329512],
         [681767,5348957],
    ]
)

strasbourg = shapely.Polygon(
    [
        [393875,5382644],
        [398185,5383652],
        [411526,5383377],
        [411434,5373887],
        [409371,5372282],
        [408179,5365681],
        [403503,5367698],
        [404236,5369761],
        [402998,5371549],
        [396901,5369944],
        [395755,5374345],
        [397864,5375446],
        [393875,5382644],
    ]
)

nantes = shapely.Polygon(
    [
         [597911,5240048],
         [619604,5239798],
         [620103,5220682],
         [598410,5220848],
         [597911,5240048],
    ]
)
newyork = shapely.Polygon(
    [
         [582100,4516163],
         [590073,4516192],
         [590013,4503800],
         [582190,4503741],
         [582100,4516163],
    ]
)

paris = shapely.Polygon(
    [
         [443272,5423320],
         [462678,5423046],
         [462472,5404120],
         [443203,5404051],
         [443272,5423320],
    ]
)

portelisabeth = shapely.Polygon(
    [
         [358931,6255259],
         [376445,6255433],
         [372154,6242906],
         [378823,6236701],
         [359569,6236469],
         [358931,6255259],
    ]
)

rio = shapely.Polygon(
    [
         [670012,7479859],
         [676732,7479713],
         [689295,7460867],
         [669939,7460429],
         [670012,7479859],
    ]
)

sanfrancisco = shapely.Polygon(
    [
         [541991,4182570],
         [553709,4185286],
         [559529,4158202],
         [543155,4157426],
         [541991,4182570],
    ]
)

shanghai = shapely.Polygon(
    [
         [344856,3486292],
         [363657,3486033],
         [363139,3442206],
         [344337,3442206],
         [344856,3486292],
    ]
)

tianjin = shapely.Polygon(
    [
         [552371,4330867],
         [571582,4330867],
         [571674,4295401],
         [552648,4295401],
         [552371,4330867],
    ]
)


scale_factor = 1/10

cities = ['Toulouse', 'Arcachon']
polygons = [toulouse, arcachon]

In [16]:
mean = np.array([0,0,0,0], dtype=np.float64)
count = np.array([0,0,0,0], dtype=np.float64)

for polygon, city in zip(polygons, cities):

    with rasterio.open(Path("/work/OT/ai4geo/DATA/DATASETS/DIGITANIE")/f"{city}/{city}.tif") as dataset:
        
        # resample data to target shape
        data = dataset.read(
            out_shape=(
                dataset.count,
                int(dataset.height * scale_factor),
                int(dataset.width * scale_factor)
            ),
            resampling=Resampling.bilinear
        )
        
        # Normalisation simple
        data = np.clip(data, 0, 1)

        # scale image transform
        transform = dataset.transform * dataset.transform.scale(
            (dataset.width / data.shape[-1]),
            (dataset.height / data.shape[-2])
        )
    
    for band in range(1, 5):
        
        stat = zonal_stats(
            polygon, 
            data[band-1],
            stats="count mean",
            affine=transform,
            band_num=band
        )[0]
        
        print(f'{city} : shape {data.shape}, band {band}', stat)
        
        count[band-1] += stat['count']
        mean[band-1] += stat['count']*(stat['mean']- mean[band-1]) / count[band-1]
    
    print(f'dataset mean: {mean}')
    print(f'dataset count: {count}')

Toulouse : shape (4, 5406, 5560), band 1 {'mean': 0.10590575567401908, 'count': 14482283}
Toulouse : shape (4, 5406, 5560), band 2 {'mean': 0.11046101122316143, 'count': 14482283}
Toulouse : shape (4, 5406, 5560), band 3 {'mean': 0.11877787155519609, 'count': 14482283}
Toulouse : shape (4, 5406, 5560), band 4 {'mean': 0.2077604062840092, 'count': 14482283}
dataset mean: [0.10590576 0.11046101 0.11877787 0.20776041]
dataset count: [14482283. 14482283. 14482283. 14482283.]
Arcachon : shape (4, 4560, 4598), band 1 {'mean': 0.052445965853460376, 'count': 14745213}
Arcachon : shape (4, 4560, 4598), band 2 {'mean': 0.07541242367946804, 'count': 14745213}
Arcachon : shape (4, 4560, 4598), band 3 {'mean': 0.09185943092175067, 'count': 14745213}
Arcachon : shape (4, 4560, 4598), band 4 {'mean': 0.0771962822781875, 'count': 14745213}
dataset mean: [0.0789354  0.09277907 0.10519757 0.14189107]
dataset count: [29227496. 29227496. 29227496. 29227496.]
